In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import re
import itertools

In [2]:
# Load results.csv file in results folder
path = Path.cwd().joinpath('results', 'result.csv')
column = ['Date_Time', 'Dataset', 'Epoch', 'Slide_Win', 'FC_Layer_Num', 'Topk']

df = pd.read_csv(path)
df

,Date_Time,Dataset,Epoch,Slide_Win,Dim,Slide_Stride,FC_Layer_Num,FC_Dim,Decay,Val_Ratio,Topk,F1 Score,Precision,Recall,Confusion Matrix,Anomalies
0,"28-Sep-2022, 12:20:28",anomaly,100,30,64,2,2,128,0.10,0.1,2,0.059182,0.032661,0.317757,"[[33604, 1007], [73, 34]]",0.0031
1,"28-Sep-2022, 12:52:18",anomaly,100,30,64,2,2,128,0.10,0.1,20,0.081119,0.047776,0.271028,"[[34033, 578], [78, 29]]",0.0031
2,"28-Sep-2022, 13:24:07",anomaly,100,30,64,2,2,128,0.10,0.1,25,0.079618,0.048077,0.233645,"[[34116, 495], [82, 25]]",0.0031
3,"28-Sep-2022, 13:56:01",anomaly,100,30,64,2,2,128,0.01,0.1,2,0.060160,0.032421,0.420561,"[[33268, 1343], [62, 45]]",0.0031
4,"28-Sep-2022, 14:27:02",anomaly,100,30,64,2,2,128,0.01,0.1,20,0.054422,0.029630,0.336449,"[[33432, 1179], [71, 36]]",0.0031
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2165,"31-Oct-2022, 04:23:13",adasyn_s,100,20,64,1,3,128,0.00,0.1,5,0.635869,0.500237,0.872388,"[[27790, 66488], [9735, 66551]]",0.4473
2166,"31-Oct-2022, 04:32:24",adasyn_s,100,5,64,1,2,128,0.00,0.1,20,0.643585,0.506317,0.882979,"[[28600, 65685], [8928, 67366]]",0.4473
2167,"31-Oct-2022, 05:49:13",adasyn_s,100,10,64,1,3,128,0.00,0.1,10,0.632842,0.491148,0.889453,"[[23976, 70305], [8434, 67859]]",0.4473
2168,"31-Oct-2022, 05:51:57",adasyn_s,100,10,64,1,3,128,0.00,0.1,5,0.639010,0.501553,0.880264,"[[27539, 66742], [9135, 67158]]",0.4473


In [3]:
# Select only Date_Time, Dataset, Epoch, Slide_Win, FC_Layer_Num and Topk
df1 = pd.DataFrame(df, columns=column)
df1

,Date_Time,Dataset,Epoch,Slide_Win,FC_Layer_Num,Topk
0,"28-Sep-2022, 12:20:28",anomaly,100,30,2,2
1,"28-Sep-2022, 12:52:18",anomaly,100,30,2,20
2,"28-Sep-2022, 13:24:07",anomaly,100,30,2,25
3,"28-Sep-2022, 13:56:01",anomaly,100,30,2,2
4,"28-Sep-2022, 14:27:02",anomaly,100,30,2,20
...,...,...,...,...,...,...
2165,"31-Oct-2022, 04:23:13",adasyn_s,100,20,3,5
2166,"31-Oct-2022, 04:32:24",adasyn_s,100,5,2,20
2167,"31-Oct-2022, 05:49:13",adasyn_s,100,10,3,10
2168,"31-Oct-2022, 05:51:57",adasyn_s,100,10,3,5


In [4]:
# df2 = df1.loc[df1['Dataset'].str.contains('_all'),:].reset_index(drop=True)
# df2[['Dataset', 'sample']] = df2['Dataset'].str.rsplit(pat='_', n=1, expand=True)
# df2.sort_values(by=['Dataset'], inplace=True)
# df2.reset_index(drop=True, inplace=True)

all_data = re.compile('msl_m_all.*|smap_m_all.*|swat_ori_all.*')
temp = []
sort_col = ['Dataset', 'Epoch', 'Slide_Win', 'FC_Layer_Num', 'Topk']

for i in range(len(df1)):
    dataset = df1.loc[i, 'Dataset']

    if all_data.match(dataset):
        temp.append(df1.loc[i, :])

df2 = pd.DataFrame(temp).sort_values(by=sort_col).reset_index(drop=True)
df2

,Date_Time,Dataset,Epoch,Slide_Win,FC_Layer_Num,Topk
0,"25-Oct-2022, 11:29:45",msl_m_all_10,100,5,2,5
1,"25-Oct-2022, 20:26:01",msl_m_all_10,100,5,2,10
2,"25-Oct-2022, 22:38:03",msl_m_all_10,100,5,2,20
3,"26-Oct-2022, 23:57:58",msl_m_all_10,100,5,3,5
4,"27-Oct-2022, 00:15:32",msl_m_all_10,100,5,3,10
...,...,...,...,...,...,...
277,"29-Oct-2022, 19:07:36",swat_ori_all_90,100,20,2,10
278,"31-Oct-2022, 02:52:58",swat_ori_all_90,100,20,2,20
279,"28-Oct-2022, 12:39:07",swat_ori_all_90,100,20,3,5
280,"30-Oct-2022, 01:34:28",swat_ori_all_90,100,20,3,10


In [5]:
# sample_dict to store completed sample for respective dataset
sample_dict = {}
sample_list = []

for dataset in df2['Dataset'].unique():
    sample_dict[dataset] = []

for i in range(len(df2)):
    dataset = df2.loc[i, 'Dataset']
    win = df2.loc[i, 'Slide_Win']
    layer = df2.loc[i, 'FC_Layer_Num']
    topk = df2.loc[i, 'Topk']
    combi = (win, layer, topk)
    
    sample_dict[dataset].append(combi)
    sample_list.append(combi)
    
df2['combi'] = sample_list
df2
    

,Date_Time,Dataset,Epoch,Slide_Win,FC_Layer_Num,Topk,combi
0,"25-Oct-2022, 11:29:45",msl_m_all_10,100,5,2,5,"(5, 2, 5)"
1,"25-Oct-2022, 20:26:01",msl_m_all_10,100,5,2,10,"(5, 2, 10)"
2,"25-Oct-2022, 22:38:03",msl_m_all_10,100,5,2,20,"(5, 2, 20)"
3,"26-Oct-2022, 23:57:58",msl_m_all_10,100,5,3,5,"(5, 3, 5)"
4,"27-Oct-2022, 00:15:32",msl_m_all_10,100,5,3,10,"(5, 3, 10)"
...,...,...,...,...,...,...,...
277,"29-Oct-2022, 19:07:36",swat_ori_all_90,100,20,2,10,"(20, 2, 10)"
278,"31-Oct-2022, 02:52:58",swat_ori_all_90,100,20,2,20,"(20, 2, 20)"
279,"28-Oct-2022, 12:39:07",swat_ori_all_90,100,20,3,5,"(20, 3, 5)"
280,"30-Oct-2022, 01:34:28",swat_ori_all_90,100,20,3,10,"(20, 3, 10)"


In [6]:
# Select unique sample and sort in ascending order
for dataset in sample_dict.keys():
    sample_dict[dataset] = sorted(list(set(sample_dict[dataset])))
    
sample_dict

{'msl_m_all_10': [(5, 2, 5),
  (5, 2, 10),
  (5, 2, 20),
  (5, 3, 5),
  (5, 3, 10),
  (5, 3, 20),
  (10, 2, 5),
  (10, 2, 10),
  (10, 2, 20),
  (10, 3, 5),
  (10, 3, 10),
  (10, 3, 20),
  (20, 2, 5),
  (20, 2, 10),
  (20, 2, 20),
  (20, 3, 5),
  (20, 3, 10),
  (20, 3, 20)],
 'msl_m_all_30': [(5, 2, 5),
  (5, 2, 10),
  (5, 2, 20),
  (5, 3, 5),
  (5, 3, 10),
  (5, 3, 20),
  (10, 2, 5),
  (10, 2, 10),
  (10, 2, 20),
  (10, 3, 5),
  (10, 3, 10),
  (10, 3, 20),
  (20, 2, 5),
  (20, 2, 10),
  (20, 2, 20),
  (20, 3, 5),
  (20, 3, 10),
  (20, 3, 20)],
 'msl_m_all_50': [(5, 2, 5),
  (5, 2, 10),
  (5, 2, 20),
  (5, 3, 5),
  (5, 3, 10),
  (5, 3, 20),
  (10, 2, 5),
  (10, 2, 10),
  (10, 2, 20),
  (10, 3, 5),
  (10, 3, 10),
  (10, 3, 20),
  (20, 2, 5),
  (20, 2, 10),
  (20, 2, 20),
  (20, 3, 5),
  (20, 3, 10),
  (20, 3, 20)],
 'msl_m_all_70': [(5, 2, 5),
  (5, 2, 10),
  (5, 2, 20),
  (5, 3, 5),
  (5, 3, 10),
  (5, 3, 20),
  (10, 2, 5),
  (10, 2, 10),
  (10, 2, 20),
  (10, 3, 5),
  (10, 3, 10),
  (1

In [7]:
stat = {}

# list of sampling rate used
win = [5, 10, 20]
layer = [2, 3]
topk = [5, 10, 20]

# Iterate through datasets
for k, v in sample_dict.items():
    
    # Compare required samples with completed samples
    for sample in itertools.product(win, layer, topk):
        status = 1 if sample in v else 0
        
        if k in stat.keys():
            stat[k].append(status)
        else:
            stat[k] = [status]

stat

{'msl_m_all_10': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'msl_m_all_30': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'msl_m_all_50': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'msl_m_all_70': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'msl_m_all_90': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'smap_m_all_10': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'smap_m_all_30': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'smap_m_all_50': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'smap_m_all_70': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'smap_m_all_90': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'swat_ori_all_10': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'swat_ori_all_30': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'swat_ori_all_50': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'swat_ori_all_70': [1, 1, 1, 1, 1, 1

In [8]:
# Convert stat to dataframe
df_stat = pd.DataFrame(stat, index=list(itertools.product(win, layer, topk)))
df_stat.to_csv('stat1.csv')
df_stat

,msl_m_all_10,msl_m_all_30,msl_m_all_50,msl_m_all_70,msl_m_all_90,smap_m_all_10,smap_m_all_30,smap_m_all_50,smap_m_all_70,smap_m_all_90,swat_ori_all_10,swat_ori_all_30,swat_ori_all_50,swat_ori_all_70,swat_ori_all_90
"(5, 2, 5)",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
"(5, 2, 10)",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
"(5, 2, 20)",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
"(5, 3, 5)",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
"(5, 3, 10)",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
"(5, 3, 20)",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
"(10, 2, 5)",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
"(10, 2, 10)",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
"(10, 2, 20)",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
"(10, 3, 5)",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [9]:
df1.loc[df1['Dataset']=='swat_all_90',:]

,Date_Time,Dataset,Epoch,Slide_Win,FC_Layer_Num,Topk
1984,"25-Oct-2022, 15:20:47",swat_all_90,100,5,2,5
1985,"25-Oct-2022, 20:18:22",swat_all_90,100,5,2,10
1986,"25-Oct-2022, 22:57:36",swat_all_90,100,5,2,20
1987,"26-Oct-2022, 01:25:40",swat_all_90,100,5,3,5
1988,"26-Oct-2022, 02:42:05",swat_all_90,100,5,3,10
1989,"26-Oct-2022, 03:35:59",swat_all_90,100,5,3,20
1990,"27-Oct-2022, 07:22:44",swat_all_90,100,10,2,5
1991,"27-Oct-2022, 15:37:45",swat_all_90,100,10,2,10
1992,"27-Oct-2022, 19:25:08",swat_all_90,100,10,2,20
1993,"27-Oct-2022, 21:20:51",swat_all_90,100,10,3,5


In [10]:
uncomplete = {}

for data in df_stat.columns:
    info = {}
    combi = df_stat.loc[df_stat[data] == 0, data].index.to_list()
    
    for i in combi:
        key = f"({i[0]}, {i[1]})"
        if key in info.keys():
            info[key].append(i[2])
        else:
            info[key] = [i[2]]

    df_info = pd.DataFrame(list(info.values()), index=list(info.keys()))
    df_info.rename(columns={0: 'first', 1: 'second', 2: 'third'}, inplace=True)
    df_info.rename_axis(data, inplace=True)
    
    uncomplete[data] = df_info.fillna('-')


In [11]:
for dataset in df_stat.columns:
    if dataset.endswith('10'):
        print()
        print(dataset.split('_')[0].upper())
        print('-'*15)
        
    print(uncomplete[dataset])
    print('_'*50)


MSL
---------------
Empty DataFrame
Columns: []
Index: []
__________________________________________________
Empty DataFrame
Columns: []
Index: []
__________________________________________________
Empty DataFrame
Columns: []
Index: []
__________________________________________________
Empty DataFrame
Columns: []
Index: []
__________________________________________________
Empty DataFrame
Columns: []
Index: []
__________________________________________________

SMAP
---------------
Empty DataFrame
Columns: []
Index: []
__________________________________________________
Empty DataFrame
Columns: []
Index: []
__________________________________________________
Empty DataFrame
Columns: []
Index: []
__________________________________________________
Empty DataFrame
Columns: []
Index: []
__________________________________________________
Empty DataFrame
Columns: []
Index: []
__________________________________________________

SWAT
---------------
Empty DataFrame
Columns: []
Index: []
_______